In [67]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime

In [68]:
url = 'http://kras.gyeongnam.go.kr/land_info/info/baseInfo/baseInfo.do'
df = pd.read_excel('./output/거제 내 풀빌라 업체 위치.xlsx')
address_list = df['주소'].values

In [76]:
def get_realestimate_info(url,df,address_list,headless = True):
    '''검색할 url과 도로명주소 목록을 넣으면 일사편리 사이트에서 정보를 조회
    parameter
    --------
    url(url) : 해당 지역의 url ex) 경남검색시, 경남url
    df(DatFrame) : 기존파일
    address_list(list) : 검색하고자하는 도로명주소
    headless(bool) : 작동창 on/off
    output
    ------
    
    '''
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
    options = Options()
    options.add_argument('--start-maximized') #big size
    options.add_argument(f'user-agent={headers}')
    if headless:
        options.add_argument('--headless') #headless
    driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

    driver.get(url) 

    index_dict = dict()
    for index, address in tqdm(enumerate(address_list)):
        try :
            save_dict = dict()
            #도로명주소 클릭
            ele = driver.find_element_by_css_selector('#searchType5')
            ele.click()
            time.sleep(1)
            all_windows = driver.window_handles 

            parent_window = all_windows[0] 
            child_window = all_windows[1] 
            driver.switch_to.window(child_window) #팝업창이동

            ele = driver.find_element_by_css_selector('#keyword')
            ele.send_keys(address)
            time.sleep(1)
            ele = driver.find_element_by_css_selector('#searchContentBox > div.search-wrap > fieldset > span > input[type=button]:nth-child(2)')
            ele.click()
            time.sleep(1)
            ele = driver.find_element_by_css_selector('#roadAddrDiv1')
            ele.click()
            time.sleep(1)
            ele = driver.find_element_by_css_selector('#resultData > div > a')
            ele.click()
            time.sleep(1)
            driver.switch_to.window(parent_window) #원래로 복귀
            time.sleep(5) #검색소요시간 

            #기본정보
            ele = driver.find_element_by_css_selector('#tab0301 > li:nth-child(1) > a')
            ele.click()

            source = driver.page_source
            soup = BeautifulSoup(source,'html.parser')
            body = soup.select('#t01')[0].text

            #임시저장
            save_dict['주소'] = address
            save_dict['면적'] = body.split('면적')[1].split('\n')[1]
            save_dict['개별공시지가'] = body.split('개별공시지가')[1].split('\n')[5].replace('\t','')
            save_dict['건물명칭'] = body.split('건물명칭')[1].split('\n')[1]
            save_dict['주용도'] = body.split('주용도')[1].split('\n')[1]
            save_dict['대지면적'] = body.split('대지면적')[1].split('\n')[1]
            save_dict['연면적'] = body.split('연면적')[1].split('\n')[1]
            save_dict['건축물수'] = body.split('건축물수')[1].split('\n')[1]
            save_dict['건출면적'] = body.split('건축면적')[1].split('\n')[1]
            save_dict['건폐율'] = body.split('건폐율')[1].split('\n')[1]
            save_dict['용적률'] = body.split('용적률')[1].split('\n')[1]
            save_dict['토지소재지'] = soup.select('#landUse_print > table:nth-of-type(1) > tbody > tr > td:nth-of-type(1)')[0].text
            save_dict['지번'] = soup.select('#landUse_print > table:nth-of-type(1) > tbody > tr > td:nth-of-type(2)')[0].text
            save_dict['지목'] = body.split('지목')[1].split('\n')[1]

            #종합저장
            index_dict[index] = save_dict
            time.sleep(2)
        except :
            try:
                WebDriverWait(driver, 5).until(EC.alert_is_present(), 'Timed out waiting for alerts to appear')
                alert = driver.switch_to.alert
                alert.accept()
                time.sleep(0.5)
                print(f"{address}:alert accepted")
            except TimeoutException:
                print(f"{address}:no alert")
            pass
    #결과저장
    result_df = pd.DataFrame(index_dict).T
    result_df = df.merge(result_df,right_on='주소',left_on = '주소',how ='left')
    result_df = result_df.drop_duplicates().reset_index(drop = True)
    today = str(datetime.today().date())
    result_df.to_excel(f'일사편리결과_{today}.xlsx',encoding='cp949',index= False)
    
    print('끝났습니다')
    driver.close()
    return result_df

In [77]:
result = get_realestimate_info(url,df, address_list,False)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: use options instead of chrome_options


경상남도 거제시 옥포대첩로 429-1_no alert

끝났습니다


### 작업본

In [66]:
sample = address_list[-5:]
result = get_realestimate_info(url,sample,False)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


no alert



KeyboardInterrupt: 

In [56]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}
options = Options()
options.add_argument('--start-maximized') #big size
options.add_argument(f'user-agent={headers}')
if headless:
    options.add_argument('--headless') #headless
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

driver.get(url) 

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [57]:
sample = address_list[-5:]

In [59]:
address =sample[1]

In [60]:
save_dict = dict()
#도로명주소 클릭
ele = driver.find_element_by_css_selector('#searchType5')
ele.click()
time.sleep(1)
all_windows = driver.window_handles 

parent_window = all_windows[0] 
child_window = all_windows[1] 
driver.switch_to.window(child_window) #팝업창이동

ele = driver.find_element_by_css_selector('#keyword')
ele.send_keys(address)
time.sleep(1)
ele = driver.find_element_by_css_selector('#searchContentBox > div.search-wrap > fieldset > span > input[type=button]:nth-child(2)')
ele.click()
time.sleep(1)
ele = driver.find_element_by_css_selector('#roadAddrDiv1')
ele.click()
time.sleep(1)
ele = driver.find_element_by_css_selector('#resultData > div > a')
ele.click()
time.sleep(1)
driver.switch_to.window(parent_window) #원래로 복귀
time.sleep(5) #검색소요시간

In [39]:
#에러 발생상황
result = driver.switch_to_alert()
result.accept()
time.sleep(2)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.alert instead
  


In [ ]:
ndex_dict = dict()
    for index, address in tqdm(enumerate(sample)):
        try :
            save_dict = dict()
            #도로명주소 클릭
            ele = driver.find_element_by_css_selector('#searchType5')
            ele.click()
            time.sleep(1)
            all_windows = driver.window_handles 

            parent_window = all_windows[0] 
            child_window = all_windows[1] 
            driver.switch_to.window(child_window) #팝업창이동

            ele = driver.find_element_by_css_selector('#keyword')
            ele.send_keys(address)
            time.sleep(1)
            ele = driver.find_element_by_css_selector('#searchContentBox > div.search-wrap > fieldset > span > input[type=button]:nth-child(2)')
            ele.click()
            time.sleep(1)
            ele = driver.find_element_by_css_selector('#roadAddrDiv1')
            ele.click()
            time.sleep(1)
            ele = driver.find_element_by_css_selector('#resultData > div > a')
            ele.click()
            time.sleep(1)
            driver.switch_to.window(parent_window) #원래로 복귀
            time.sleep(5) #검색소요시간 

            #기본정보
            ele = driver.find_element_by_css_selector('#tab0301 > li:nth-child(1) > a')
            ele.click()

            source = driver.page_source
            soup = BeautifulSoup(source,'html.parser')
            body = soup.select('#t01')[0].text

            #임시저장
            save_dict['주소'] = address
            save_dict['면적'] = body.split('면적')[1].split('\n')[1]
            save_dict['개별공시지가'] = body.split('개별공시지가')[1].split('\n')[5].replace('\t','')
            save_dict['건물명칭'] = body.split('건물명칭')[1].split('\n')[1]
            save_dict['주용도'] = body.split('주용도')[1].split('\n')[1]
            save_dict['대지면적'] = body.split('대지면적')[1].split('\n')[1]
            save_dict['연면적'] = body.split('연면적')[1].split('\n')[1]
            save_dict['건축물수'] = body.split('건축물수')[1].split('\n')[1]
            save_dict['건출면적'] = body.split('건축면적')[1].split('\n')[1]
            save_dict['건폐율'] = body.split('건폐율')[1].split('\n')[1]
            save_dict['용적률'] = body.split('용적률')[1].split('\n')[1]
            save_dict['토지소재지'] = soup.select('#landUse_print > table:nth-of-type(1) > tbody > tr > td:nth-of-type(1)')[0].text
            save_dict['지번'] = soup.select('#landUse_print > table:nth-of-type(1) > tbody > tr > td:nth-of-type(2)')[0].text
            save_dict['지목'] = body.split('지목')[1].split('\n')[1]

            #종합저장
            index_dict[index] = save_dict
            time.sleep(2)
        except :
            result = driver.switch_to_alert()
            result.accept()
            pass
    #결과저장
    result_df = pd.DataFrame(index_dict).T
    df = df.merge(result_df,right_on='도로명주소',left_on = '주소',how ='left')
    df = df.drop_duplicates().reset_index(drop = True)
    today = str(datetime.today().date())
    df.to_excel(f'일사편리결과_{today}.xlsx',encoding='cp949',index= False)
    
    print('끝났습니다')
    driver.close()
    return df